In [1]:
#hide
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#hide
%load_ext autoreload
%autoreload 2

%matplotlib inline
%cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [ ]:
__all__ ='ImageToImageList get_dataloader'.split(" ")

In [ ]:
#hide
# !git clone https://github.com/prajwal-suresh13/dl_lib.git

In [3]:
from dl_lib.core.all import *

In [ ]:
class ImageToImageList():
    def __init__(self, imagesA, imagesB, lenA, lenB):
        self.imagesA, self.imagesB = imagesA, imagesB
        self.lenA, self.lenB = lenA, lenB

    @classmethod
    def from_folders(cls, pathA, pathB, extensions=None, recurse=False,include=None, tfms=None):
        imagesA = ImageList.from_folder(pathA, extensions=extensions, recurse=recurse, include=include, tfms=tfms)
        imagesB = ImageList.from_folder(pathB, extensions=extensions, recurse=recurse, include=include, tfms=tfms)
        lenA,lenB =len(imagesA), len(imagesB)
        return cls(imagesA,imagesB,lenA, lenB)

    def __getitem__(self,idx):
        if isinstance(idx, slice):
            indexes =list(range(ifnone(idx.start, 0),idx.stop,ifnone(idx.step,1)))
            imgsA = [self.imagesA[i%self.lenA] for i in indexes]
            imgsB = [self.imagesB[random.randint(0,self.lenB-1)] for i in indexes]
        elif isinstance(idx,int):
            imgsA = self.imagesA[idx%self.lenA]
            imgsB = self.imagesB[random.randint(0,self.lenB-1)]
        else:
            imgsA=[self.imagesA[i%self.lenA] for i in idx]
            imgsB=[self.imagesB[random.randint(0,self.lenB-1)] for _ in idx]
        return imgsA, imgsB

    def __repr__(self):
        return f'{self.__class__.__name__}\nImagesA\n: {self.imagesA}\nImagesB\n: {self.imagesB}\n'

    def __len__(self):
        return max(self.lenA, self.lenB)



In [ ]:
#hide
pathA = Path('./image_colorization_cyclegan/data/bw_frames')
pathB = Path('./image_colorization_cyclegan/data/color_frames')

In [ ]:
#hide
il = ImageToImageList.from_folders(pathA, pathB)

In [ ]:
#hide
il

ImageToImageList
ImagesA
: ImageList (3000 items) 
[Path('image_colorization_cyclegan/data/bw_frames/3968.jpg'), Path('image_colorization_cyclegan/data/bw_frames/3969.jpg'), Path('image_colorization_cyclegan/data/bw_frames/3970.jpg'), Path('image_colorization_cyclegan/data/bw_frames/3973.jpg'), Path('image_colorization_cyclegan/data/bw_frames/3975.jpg'), Path('image_colorization_cyclegan/data/bw_frames/3976.jpg'), Path('image_colorization_cyclegan/data/bw_frames/3977.jpg'), Path('image_colorization_cyclegan/data/bw_frames/3978.jpg'), Path('image_colorization_cyclegan/data/bw_frames/3980.jpg'), Path('image_colorization_cyclegan/data/bw_frames/3982.jpg')...]
Path : image_colorization_cyclegan/data/bw_frames 
 Transforms : None
ImagesB
: ImageList (3000 items) 
[Path('image_colorization_cyclegan/data/color_frames/3986.jpg'), Path('image_colorization_cyclegan/data/color_frames/3989.jpg'), Path('image_colorization_cyclegan/data/color_frames/3990.jpg'), Path('image_colorization_cyclegan/data

In [ ]:
#hide
PIL.Image.open(pathA.ls()[0]).mode,PIL.Image.open(pathB.ls()[0]).mode

('RGB', 'RGB')

In [ ]:
def get_dataloader(pathA, pathB, extensions=None, recurse=False, include=None,tfms=None, image_size=128, batch_size=1, shuffle=True,num_workers=2,pin_memory=True,**kwargs):
    stfms=[MakeRGB(), ResizeFixed(image_size), to_byte_tensor, to_float_tensor]
    if tfms is not None: stfms += tfms
    il = ImageToImageList.from_folders(pathA, pathB, extensions=extensions, recurse=recurse, include=include, tfms=stfms)
    return DataLoader(il, batch_size=batch_size, shuffle=shuffle,num_workers=num_workers,pin_memory=pin_memory,**kwargs)

In [ ]:
#hide
d = get_dataloader(pathA, pathB, batch_size=4, image_size=1024, shuffle=False)

In [ ]:
#hide
xb,yb = next(iter(d))

In [ ]:
#hide
xb[0].shape, yb[0].shape

(torch.Size([3, 1024, 1024]), torch.Size([3, 1024, 1024]))

In [ ]:
#hide
show_img2img_batch(d, 2)

Output hidden; open in https://colab.research.google.com to view.

In [4]:
#hide
!pip install fire
!python dl_lib/notebook2script.py image_colorization/cyclegan/datadl.ipynb dl_lib/cyclegan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.5 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=2c364eddfbeb5e06b413433c54c445a0a3d95143a31b8ab4b360f8fab44580ac
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
Converted image_colorization/cyclegan/datadl.ipynb to dl_lib/cyclegan/datadl.py
